In [5]:
import numpy as np
import matplotlib.pyplot as plt
import time

class KalmanFairPriceEstimator:
    def __init__(self, initial_mid_price, Q=0.09, R=0.04):
        self.F = 1  # Matriz de transición de estado
        self.H = 1  # Matriz de observación
        self.Q = Q  # Varianza del ruido del proceso
        self.R = R  # Varianza del ruido de medición

        # Inicialización del filtro
        self.x = initial_mid_price  # Estimación inicial del fair price
        self.P = 1  # Covarianza inicial del error de estimación

    def update(self, new_bid, new_ask):
        new_mid_price = (new_bid + new_ask) / 2

        # Paso de predicción
        x_pred = self.F * self.x
        P_pred = self.F * self.P * self.F + self.Q

        # Paso de actualización
        K = P_pred * self.H / (self.H * P_pred * self.H + self.R)
        self.x = x_pred + K * (new_mid_price - self.H * x_pred)
        self.P = (1 - K * self.H) * P_pred

        return self.x




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

# Asegurarnos de usar el backend correcto
import matplotlib
matplotlib.use('TkAgg')

class KalmanFairPriceEstimator:
    def __init__(self, initial_mid_price, Q=0.09, R=0.04):
        self.F = 1  # Matriz de transición de estado
        self.H = 1  # Matriz de observación
        self.Q = Q  # Varianza del ruido del proceso
        self.R = R  # Varianza del ruido de medición

        # Inicialización del filtro
        self.x = initial_mid_price  # Estimación inicial del fair price
        self.P = 1  # Covarianza inicial del error de estimación

    def update(self, new_bid, new_ask):
        new_mid_price = (new_bid + new_ask) / 2

        # Paso de predicción
        x_pred = self.F * self.x
        P_pred = self.F * self.P * self.F + self.Q

        # Paso de actualización
        K = P_pred * self.H / (self.H * P_pred * self.H + self.R)
        self.x = x_pred + K * (new_mid_price - self.H * x_pred)
        self.P = (1 - K * self.H) * P_pred

        return self.x

# Cerrar todas las figuras anteriores
plt.close('all')

# Configuración inicial
WINDOW_SIZE = 50  # Número de puntos a mostrar en la ventana
initial_bid, initial_ask = 100, 102
initial_mid_price = (initial_bid + initial_ask) / 2
estimator = KalmanFairPriceEstimator(initial_mid_price, Q=0.09, R=0.04)

# Inicializar las listas con el primer punto
true_fair_prices = [initial_mid_price]
fair_price_estimates = [initial_mid_price]
bid_prices = [initial_bid]
ask_prices = [initial_ask]
time_steps = [0]

# Configurar la gráfica
plt.ion()  # Activar modo interactivo
fig, ax = plt.subplots(figsize=(12, 6))
line_true, = ax.plot(time_steps, true_fair_prices, 'g-', alpha=0.5, label='True Fair Price')
line_est, = ax.plot(time_steps, fair_price_estimates, 'r-', linewidth=2, label='Fair Price Estimado (Kalman)')
line_bid, = ax.plot(time_steps, bid_prices, 'b--', alpha=0.7, label='Best Bid')
line_ask, = ax.plot(time_steps, ask_prices, 'y--', alpha=0.7, label='Best Ask')

# Configurar la gráfica
ax.set_title(f'Mid Price: {initial_mid_price:.2f} | Kalman Fair Price: {initial_mid_price:.2f}', pad=20)
ax.set_xlabel('Tiempo')
ax.set_ylabel('Precio')
ax.legend(loc='upper left')
ax.grid(True)

# Establecer límites iniciales
ax.set_ylim(95, 115)
ax.set_xlim(0, WINDOW_SIZE)

# Mostrar la figura antes de empezar
plt.show()

# Simular la llegada de datos en tiempo real
try:
    for i in range(1, 200):
        # Simular nuevos datos de bid y ask
        true_price = 100 + 0.05 * i + np.random.normal(0, 0.5)
        bid = true_price - np.abs(np.random.normal(0, 0.3))
        ask = true_price + np.abs(np.random.normal(0, 0.3))
        mid_price = (bid + ask) / 2

        # Actualizar el estimador
        fair_price = estimator.update(bid, ask)

        # Actualizar las listas de datos
        true_fair_prices.append(true_price)
        fair_price_estimates.append(fair_price)
        bid_prices.append(bid)
        ask_prices.append(ask)
        time_steps.append(i)
        
        if len(time_steps) > WINDOW_SIZE:
            true_fair_prices.pop(0)
            fair_price_estimates.pop(0)
            bid_prices.pop(0)
            ask_prices.pop(0)
            time_steps.pop(0)

        # Actualizar la gráfica
        line_true.set_data(time_steps, true_fair_prices)
        line_est.set_data(time_steps, fair_price_estimates)
        line_bid.set_data(time_steps, bid_prices)
        line_ask.set_data(time_steps, ask_prices)
        
        # Actualizar título y límites
        ax.set_title(f'Bid: {bid:.2f} | Ask: {ask:.2f} | Kalman Fair Price: {fair_price:.2f}', pad=20)
        ax.set_xlim(max(0, i - WINDOW_SIZE), i + 5)
        
        # Forzar la actualización de la visualización
        plt.pause(0.01)  # Este comando actualiza la visualización

except KeyboardInterrupt:
    print("\nSimulación interrumpida por el usuario")
finally:
    plt.ioff()
    plt.show()